In [1]:
import time
import sqlite3
import pandas as pd
import pyarrow as pa

# 1. Load Data into SQLite Database. 

Note: Once this step is done, and the sqlite database is created, you can skip right to step 2 in the future

In [10]:
%%time

# load doi_sdg data
sdg = pd.read_csv('doi_sdg.csv', delimiter=',')
sdg.doi = sdg.doi.astype("string").str.lower()
sdg.tail()

CPU times: user 4.82 s, sys: 675 ms, total: 5.5 s
Wall time: 29.2 s


,doi,category_sdg_code
5791135,10.1007/978-3-319-57165-2_5,3
5791136,10.1080/02763869.2012.641855,4
5791137,10.1109/tap.2011.2163479,3
5791138,10.22190/jtesap1904453t,16
5791139,10.1177/1367549419871342,16


In [11]:
import numpy as np
len(np.unique(sdg.doi))

5481839

In [9]:
%%time

# load SciNet_Papers_V0-1 data
scinet = pd.read_csv("SciNet_Papers_V0-1.csv", sep="\t")
scinet.dropna(subset=['DOI'], inplace=True)
scinet.DOI = scinet.DOI.astype('string')
scinet.DOI = scinet.DOI.str.lower()
scinet.head()

CPU times: user 3min 33s, sys: 37.5 s, total: 4min 11s
Wall time: 4min 11s


,MAGID,DOI,DocType,Year,Date,JID,ConfSID,ConfInsID,C_all,C10,R,C3,C5,NIH_count,Patent_count,Newsfeed_count,Twitter_count,NCT_count
4,14558443,10.1007/978-3-662-45174-8_28,Conference,2014.0,2014-10-12,NaN,1.131603e+09,4038532.0,14.0,14.0,11.0,6.0,9.0,NaN,NaN,NaN,NaN,NaN
5,15354235,10.1007/978-3-662-44777-2_60,Conference,2014.0,2014-09-08,NaN,1.154039e+09,157008481.0,9.0,9.0,18.0,6.0,8.0,NaN,NaN,NaN,NaN,NaN
6,16763247,10.22004/ag.econ.174942,Repository,2014.0,2014-01-01,3.121261e+09,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
10,24327294,10.1007/978-3-319-03973-2_13,Conference,2013.0,2013-01-01,NaN,1.196984e+09,NaN,20.0,20.0,25.0,12.0,17.0,NaN,NaN,NaN,NaN,NaN
15,35428871,10.1023/a:1022270923160,Journal,1998.0,1998-01-01,2.016092e+08,NaN,NaN,6.0,2.0,16.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [10]:
# connect to sqlite database
con = sqlite3.connect("sdg.sqlite")
cur = con.cursor()

In [ ]:
%%time

# create doi_sdg table and index
table_name = "doi_sdg"
sdg.to_sql(table_name, con, index=False)
cur.execute(f"create index {table_name}_idx on {table_name}(doi)");

In [ ]:
%%time

# create scinet_papers table and index
table_name = "scinet_papers"
scinet.to_sql(table_name, con, index=False)
cur.execute(f"create index {table_name}_idx on {table_name}(doi)");

In [15]:
%%time

# create doi_sdg_concat table and index
sql = """
select doi, group_concat(category_sdg_code) as sdg_codes
from doi_sdg
group by doi
"""
table_name = "doi_sdg_concat"
df_sdg_concat = pd.read_sql(sql, con)
df_sdg_concat.to_sql(table_name, con, index=False)
cur.execute(f"create index {table_name}t_idx on {table_name}(doi)")

CPU times: user 19.6 s, sys: 23.1 s, total: 42.6 s
Wall time: 46.2 s


In [ ]:
%%time

# create scinet_papers_matched and index
sql = """
select s.*, d.sdg_codes
from scinet_papers as s
inner join doi_sdg_concat as d
on s.doi = d.doi
"""
table_name = "scinet_papers_matched"
df_matched = pd.read_sql(sql, con)
df_matched.to_sql(table_name, con)
cur.execute(f"create index {table_name}_idx on {table_name}(doi)")

## Don't run the codes above

# 2. Restore Table from Database

In [2]:
import sqlite3

In [3]:
import sqlite3
con = sqlite3.connect("sdg.sqlite")
cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('doi_sdg',), ('scinet_papers',), ('doi_sdg_concat',), ('scinet_papers_matched',)]


**Concat: Aggregated SDG codes (list)**

**SciNet_Matched: Table we are interested in**

In [43]:
cur.execute("PRAGMA table_info('scinet_papers_matched')").fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'MAGID', 'INTEGER', 0, None, 0),
 (2, 'DOI', 'TEXT', 0, None, 0),
 (3, 'DocType', 'TEXT', 0, None, 0),
 (4, 'Year', 'REAL', 0, None, 0),
 (5, 'Date', 'TEXT', 0, None, 0),
 (6, 'JID', 'REAL', 0, None, 0),
 (7, 'ConfSID', 'REAL', 0, None, 0),
 (8, 'ConfInsID', 'REAL', 0, None, 0),
 (9, 'C_all', 'REAL', 0, None, 0),
 (10, 'C10', 'REAL', 0, None, 0),
 (11, 'R', 'REAL', 0, None, 0),
 (12, 'C3', 'REAL', 0, None, 0),
 (13, 'C5', 'REAL', 0, None, 0),
 (14, 'NIH_count', 'REAL', 0, None, 0),
 (15, 'Patent_count', 'REAL', 0, None, 0),
 (16, 'Newsfeed_count', 'REAL', 0, None, 0),
 (17, 'Twitter_count', 'REAL', 0, None, 0),
 (18, 'NCT_count', 'REAL', 0, None, 0),
 (19, 'sdg_codes', 'TEXT', 0, None, 0)]

In [6]:
import pandas as pd

In [46]:
df_matched2 = pd.read_sql("select * from scinet_papers_matched", con)

In [49]:
df_matched2.head()

,index,MAGID,DOI,DocType,Year,Date,JID,ConfSID,ConfInsID,C_all,C10,R,C3,C5,NIH_count,Patent_count,Newsfeed_count,Twitter_count,NCT_count,sdg_codes
0,0,368915168,10.4070/kcj.2015.45.3.202,Journal,2015.0,2015-05-01,1.639715e+08,NaN,NaN,67.0,67.0,49.0,24.0,55.0,NaN,NaN,NaN,NaN,NaN,3
1,1,1480729583,10.1017/s1365100517000293,Journal,2017.0,2017-09-06,1.367455e+08,NaN,NaN,11.0,11.0,99.0,5.0,11.0,NaN,NaN,NaN,NaN,NaN,8
2,2,1481854986,10.1057/9780230242234_7,Repository,2009.0,2009-01-01,3.121261e+09,NaN,NaN,0.0,0.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,7
3,3,1488332647,10.1109/isbmsb.2008.4536671,Conference,2008.0,2008-06-03,NaN,2.623662e+09,2.626942e+09,4.0,4.0,3.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,7
4,4,1500085194,10.1109/irecon.1964.1147337,Conference,1964.0,1964-01-01,NaN,2.755067e+09,NaN,2.0,2.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,7


In [26]:
len(df_matched2)

5032587

In [12]:
5032587 / 5481839

0.9180472100694675

In [13]:
## About 91% of articles from SDG datasets are matched

In [38]:
len(df_matched2)/ len(scinet)
## Only 3% articles are related to SDG

0.0375204463326804

In [25]:
df_matched2[df_matched2.sdg_codes.apply(len)>2]

,index,MAGID,DOI,DocType,Year,Date,JID,ConfSID,ConfInsID,C_all,C10,R,C3,C5,NIH_count,Patent_count,Newsfeed_count,Twitter_count,NCT_count,sdg_codes
19,19,1927641356,10.1007/s10584-015-1505-0,Journal,2017.0,2017-09-01,2.047995e+08,NaN,NaN,54.0,50.0,29.0,37.0,50.0,NaN,NaN,NaN,1.0,NaN,"12,13"
35,35,1976945638,10.1353/jda.2012.0025,Journal,2012.0,2012-10-01,1.456782e+07,NaN,NaN,2.0,2.0,35.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,"3,10"
48,48,1986106498,10.5296/jmr.v5i2.3266,Journal,2013.0,2013-04-07,2.764717e+09,NaN,NaN,33.0,33.0,29.0,8.0,14.0,NaN,NaN,NaN,NaN,NaN,"3,8"
109,109,2087223873,10.1016/0960-1481(96)00076-6,Journal,1997.0,1997-02-01,8.153010e+05,NaN,NaN,3.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,"7,13"
118,118,2106682816,10.17159/1727-3781/2012/v15i3a2504,Journal,2012.0,2012-01-01,2.739045e+09,NaN,NaN,3.0,3.0,7.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,"5,16"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5032526,5032526,2060816285,10.1080/00219266.2000.9655721,Journal,2000.0,2000-09-01,1.859549e+08,NaN,NaN,1.0,1.0,7.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,"7,13"
5032554,5032554,2734534492,10.5430/ijfr.v8n3p162,Journal,2017.0,2017-06-12,2.764951e+09,NaN,NaN,2.0,2.0,9.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,"1,10"
5032564,5032564,2885574959,10.5840/wcp232018511074,BookChapter,2018.0,2018-05-31,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,"1,2"
5032573,5032573,2994724080,10.3390/su11247179,Journal,2019.0,2019-12-01,1.013438e+07,NaN,NaN,0.0,0.0,29.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,"4,11"


In [27]:
from collections import Counter

In [28]:
Counter(df_matched2.DocType)

Counter({'Journal': 3587356,
         'Repository': 80867,
         'Conference': 370715,
         'BookChapter': 80272,
         None: 894942,
         'Book': 18104,
         'Thesis': 320,
         'Dataset': 11})

In [36]:
3587356/len(df_matched2)

0.7128254315325299

In [30]:
scinet = pd.read_csv("SciNet_Papers_V0-1.csv", sep="\t")


In [32]:
Counter(scinet.DocType)

Counter({'Journal': 90764813,
         'Conference': 5123597,
         'Repository': 3083949,
         nan: 20451018,
         'Book': 4629342,
         'BookChapter': 3932366,
         'Thesis': 5998509,
         'Dataset': 145594})

In [34]:
90764813/len(scinet)

0.676696954282613

In [ ]:
scinet.